In [2]:
import numpy as np
import glob
import cv2
import os
import pandas as pd

In [18]:
caminho = "/content"  # Diretório que deseja listar os arquivos
lista_arquivos = os.listdir(caminho)  # Lista todos os arquivos no diretório

nomes_arquivos = []  # Lista para armazenar somente os nomes dos arquivos

for arquivo in lista_arquivos:
    if os.path.isfile(os.path.join(caminho, arquivo)):
        nomes_arquivos.append(arquivo)


['apple3.jpg', 'apple_green.jpg', 'apple2.jpg', 'apple1.jpg', 'pineapple.jpg']


In [20]:
index = {}
images = {}

for imagePath in nomes_arquivos:

  image = cv2.imread(imagePath)
  images[imagePath] = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  hist = cv2.calcHist([image], [1,2], None, [8,8], [0, 256, 0, 256])
  hist = cv2.normalize(hist, hist).flatten()
  index[imagePath] = hist


Função compareHist

A flag pode ser qualquer um dos seguintes método:

cv2.HISTCMP_CORREL: Calcula a correlação entre os dois histogramas.

cv2.HISTCMP_CHISQR: Aplica a distância Qui-Quadrado aos histogramas.

cv2.HISTCMP_INTERSECT: Calcula a interseção entre dois histogramas.

cv2.HISTCMP_BHATTACHARYYA: Distância de Bhattacharyya, usada para medir a “sobreposição” entre os dois histogramas.

In [23]:
OPENCV_METHODS = (
    ("Correlation", cv2.HISTCMP_CORREL),
    ("Chi-Squared", cv2.HISTCMP_CHISQR),
    ("Intersection", cv2.HISTCMP_INTERSECT),
    ("Bhattacharyya", cv2.HISTCMP_BHATTACHARYYA))

imagem_analisada = 'apple1.jpg'
        
lista_resultados = []
lista_methodName = []

for (methodName, method) in OPENCV_METHODS:

    results = {}
    reverse = False

    # se estivermos usando a Correlation ou Intersection
    # classificar os resultados na ordem inversa
    if methodName in ("Correlation", "Intersection"):
        reverse = True

    for (k, hist) in index.items():
        # Calcular a distancia entre os dois histogramas usando os metodos
        # Atualizar o dicionario de resultados
        d = cv2.compareHist(index[imagem_analisada], hist, method)
        results[k] = d

    # Ordenar os resultados
    #print(methodName)
    lista_methodName.append(methodName)
    results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)
    lista_resultados.append(results)

In [24]:
# Criar o dataframe

for i in range(len(lista_methodName)):
    lista_resultados[i] =  pd.DataFrame(lista_resultados[i])
    lista_resultados[i]['Metodo'] = lista_methodName[i]


df = pd.concat(lista_resultados)
df.to_csv('Resultado_compareHist.csv', sep=';', encoding='latin1')

In [25]:
#Bhattacharyya
Bhattacharyya = df.loc[df['Metodo'] == 'Bhattacharyya']
Bhattacharyya.sort_values(0)

for i in range(len(Bhattacharyya)):
    print(Bhattacharyya[1][i])

apple1.jpg
apple2.jpg
pineapple.jpg
apple3.jpg
apple_green.jpg
